In [0]:
%pip install -r bundles/requirements.txt
%pip install ./artifacts/monailabel-0.8.5-py3-none-any.whl --no-deps
%pip install monai==1.4.0 pytorch-ignite --no-deps
%pip install databricks-sdk==0.56 --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
from types import SimpleNamespace

def init_monai_widgets_environment():

  dbutils.widgets.text("table", "main.pixels_solacc.object_catalog", label="1.0 Catalog Schema Table to store object metadata into")
  table = dbutils.widgets.get("table")

  dbutils.widgets.text("sqlWarehouseID", "", label="2.0 SQL Warehouse")
  sql_warehouse_id = dbutils.widgets.get("sqlWarehouseID")

  if sql_warehouse_id == "":
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()
    sql_warehouse = w.warehouses.list()[0]
    sql_warehouse_id = sql_warehouse.id
    dbutils.widgets.text("sqlWarehouseID", sql_warehouse_id, label="2.0 SQL Warehouse")
    print(f"SQL Warehouse is mandatory, taking the first available: '{sql_warehouse.name}'")

    dbutils.widgets.text("volume", "main.pixels_solacc.pixels_volume", label="3.0 Catalog Schema Volume where pixel volumes are stored into")
    volume = dbutils.widgets.get("volume")

    dbutils.widgets.text("model_uc_name", "main.pixels_solacc.monai_pixels_model", label="3.1 Model name stored in UC")
    model_uc_name = dbutils.widgets.get("model_uc_name")
    dbutils.widgets.text("serving_endpoint_name", "pixels-monai-uc", label="4.0 Serving Endpoint name")
    serving_endpoint_name = dbutils.widgets.get("serving_endpoint_name")

#    sql_warehouse_id, table, volume = init_widgets(show_volume=True)
#    model_uc_name, serving_endpoint_name = init_model_serving_widgets()

    dbutils.widgets.text("use_service_principal", "False", label="5.0 Use Service Principal")
    use_service_principal = dbutils.widgets.get("use_service_principal").lower() == "true"

    volume_path = volume.replace(".","/")

    dbutils.widgets.text("model_name", "wholeBody_ct_segmentation", label="6.0 Model Name")
    model_name = dbutils.widgets.get("model_name")

    dbutils.widgets.text("dest_dir", f"/Volumes/{volume_path}/monai_serving/{model_name}/", label="7.0 Destination Folder")
    dest_dir = dbutils.widgets.get("dest_dir")

    # Pick one of the series_uid available in the pixels' catalog table
    dbutils.widgets.text("series_uid", "1.2.156.14702.1.1000.16.1.2020031111365289000020001", label="8. Example Series UID")
    series_uid = dbutils.widgets.get("series_uid")

    # Bulk Inferencing Result Table
    dbutils.widgets.text("inference_results_table", f"{table}_{model_name}_results", label="9. Inference Results")
    inference_results_table = dbutils.widgets.get("inference_results_table")
    
    #init_env()
#    sql_warehouse_id = dbutils.widgets.get("sqlWarehouseID")
#    table = dbutils.widgets.get("table")

    if not spark.catalog.tableExists(table):
        raise Exception("The configured table does not exist!")

    os.environ["DATABRICKS_TOKEN"] = ctx.apiToken
    os.environ["DATABRICKS_WAREHOUSE_ID"] = sql_warehouse_id
    os.environ["DATABRICKS_HOST"] = ctx.apiUrl
    os.environ["DATABRICKS_PIXELS_TABLE"] = table
    os.environ["DEST_DIR"] = dest_dir


    return SimpleNamespace(**{
        "sql_warehouse_id": sql_warehouse_id,
        "table": table,
        "volume": volume,
        "model_uc_name": model_uc_name,
        "serving_endpoint_name": serving_endpoint_name,
        "use_service_principal": use_service_principal,
        "model_name": model_name,
        "dest_dir": dest_dir,
        "series_uid":series_uid,
        "inference_results_table": inference_results_table
    })



In [0]:
init_monai_widgets_environment()